<a href="https://colab.research.google.com/github/xhxuciedu/CS175/blob/master/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


In [0]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [4]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2110
Epoch [1/5], Step [200/600], Loss: 2.0996
Epoch [1/5], Step [300/600], Loss: 1.9830
Epoch [1/5], Step [400/600], Loss: 1.8444
Epoch [1/5], Step [500/600], Loss: 1.8538
Epoch [1/5], Step [600/600], Loss: 1.7486
Epoch [2/5], Step [100/600], Loss: 1.6980
Epoch [2/5], Step [200/600], Loss: 1.6032
Epoch [2/5], Step [300/600], Loss: 1.6528
Epoch [2/5], Step [400/600], Loss: 1.5775
Epoch [2/5], Step [500/600], Loss: 1.5262
Epoch [2/5], Step [600/600], Loss: 1.4600
Epoch [3/5], Step [100/600], Loss: 1.3811
Epoch [3/5], Step [200/600], Loss: 1.4394
Epoch [3/5], Step [300/600], Loss: 1.3322
Epoch [3/5], Step [400/600], Loss: 1.2999
Epoch [3/5], Step [500/600], Loss: 1.3125
Epoch [3/5], Step [600/600], Loss: 1.1545
Epoch [4/5], Step [100/600], Loss: 1.2165
Epoch [4/5], Step [200/600], Loss: 1.0194
Epoch [4/5], Step [300/600], Loss: 1.1824
Epoch [4/5], Step [400/600], Loss: 1.0488
Epoch [4/5], Step [500/600], Loss: 1.0878
Epoch [4/5], Step [600/600], Loss:

In [5]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the model on the 10000 test images: 82 %
